In [1]:
import eikon as ek
import pandas as pd
from fetching_files import *
ek.set_app_key("5f62315c837645a78ff033f35644b3ba3ef5753a")
from importlib import reload

In [2]:
# Reloading the 'open_and_clean' module and re-importing
import preprocess.file_preprocess.open_and_clean
reload(preprocess.file_preprocess.open_and_clean)
from preprocess.file_preprocess.open_and_clean import *

# Reloading the 'filter_out_relevant_data' module and re-importing
import preprocess.file_preprocess.filter_out_relevant_data
reload(preprocess.file_preprocess.filter_out_relevant_data)
from preprocess.file_preprocess.filter_out_relevant_data import *

# Reloading the 'preprocess_to_df' module and re-importing
import preprocess.file_preprocess.preprocess_to_df
reload(preprocess.file_preprocess.preprocess_to_df)
from preprocess.file_preprocess.preprocess_to_df import *

# Reloading the 'find_and_remove_duplicates' module and re-importing
import preprocess.file_preprocess.find_and_remove_duplicates
reload(preprocess.file_preprocess.find_and_remove_duplicates)
from preprocess.file_preprocess.find_and_remove_duplicates import *

# Reloading the 'getters_and_counts' module and re-importing
import metricsUtils.getters_and_counts
reload(metricsUtils.getters_and_counts)
from metricsUtils.getters_and_counts import *

import metricsUtils.calculate_new_metrics
reload(metricsUtils.calculate_new_metrics)
from metricsUtils.calculate_new_metrics import *

#Reloading the 'segment_pipeline' module and re-importing
import pipelines.segment_pipeline
reload(pipelines.segment_pipeline)
from pipelines.segment_pipeline import *

import preprocess.configurations.configurations
reload(preprocess.configurations.configurations)
from preprocess.configurations.configurations import * 

import firm 
reload(firm)
from firm import Firm

import metricsUtils.impuding_calculations
reload(metricsUtils.impuding_calculations)
from metricsUtils.impuding_calculations import *

In [4]:
def find_wacc(year): 
    df = pd.read_csv(rf"non_ren_years_{-year}.csv")
    roe = pd.read_csv(rf"alt_industry_roe\industry_roe_for_year_-{year}y.csv")
    columns_to_check = ['Revenue', 'Company Market Cap', 'Total Debt', 'Price Close', 'Common Shares - Outstanding - Total', 'Total Assets', "Interest Expense",  "Interest Bearing Liabilities - Total"]
    df_new = df.dropna(subset=columns_to_check)
    df_new = df_new[(df_new['Total Assets'] != 0) & (df_new["Revenue"] > 20000000)] 
    df_new2 = fill_nan_wacc_tax_rate(df_new)
    df_FEPS = df_new2[((~df_new2['Earnings Per Share - SmartEstimate®'].isna())&(df_new2['Earnings Per Share - SmartEstimate®'])!=0 ) & ((df_new2['Earnings Per Share - SmartEstimate®.1']!=0)&(~df_new2['Earnings Per Share - SmartEstimate®.1'].isna()))]
    roe_estimates = calculate_median_industry_roe(roe, "TRBC Activity Code")
    firm_objects = create_firm_objects(df_FEPS, roe_estimates, 'TRBC Activity Code')

    wacc = []
    coe = []
    cod = []
    firm_names = []
    non_converging_firms = []

    # Iterate through each firm object
    for firm_name, firm in firm_objects.items():
        coe_result = firm.find_cost_of_equity()

        if coe_result is None:
            non_converging_firms.append(firm_name)
            wacc_result = None  # Or set a default value
            coe_result = None  # Or set a default value
        else:
            wacc_result = firm.calculate_gls_cost_of_capital()

        wacc.append(wacc_result)
        coe.append(coe_result)
        cod.append(firm.calculate_cost_of_debt())
        firm_names.append(firm_name)

    # Create a DataFrame from the results
    wacc_df = pd.DataFrame({
        "Firm Name": firm_names, 
        "WACC": wacc, 
        "COE": coe, 
        "COD": cod
    })
    return wacc_df

for i in range(0,20): 
    if i == 10: 
        print("hello")
        wacc_df = find_wacc(i)
    wacc_df = find_wacc(i)
    print(wacc_df)
    wacc_df.to_csv(rf"non_ren_wacc_for_time_-{i}y.csv", index=False)



      Firm Name      WACC       COE       COD
0        BRKa.N  0.046316  0.085626  0.035456
1          GE.N  0.023084  0.128125  0.002286
2         MMM.N  0.030286  0.070359  0.028830
3        TECK.N  0.056859  0.116484  0.001456
4         WMT.N  0.027378  0.068324  0.047689
...         ...       ...       ...       ...
2015    1610.HK  0.055320  0.103000  0.024298
2016  300709.SZ  0.037735  0.060875  0.017476
2017     6532.T  0.067716  0.088375  0.009602
2018     4189.T  0.046830  0.102531  0.003834
2019  300769.SZ  0.034337  0.085781  0.020869

[2020 rows x 4 columns]
      Firm Name      WACC       COE       COD
0        BRKa.N  0.049355  0.091729  0.035456
1          GE.N  0.024285  0.134859  0.002286
2         MMM.N  0.031765  0.075027  0.028830
3        TECK.N  0.070527  0.144578  0.001456
4         WMT.N  0.027768  0.069559  0.047689
...         ...       ...       ...       ...
2015    1610.HK  0.058904  0.111000  0.024298
2016  300709.SZ  0.037218  0.059937  0.017476
2017     

FileNotFoundError: [Errno 2] No such file or directory: 'non_ren_years_-7.csv'

In [5]:
coc = {}


for i in range(7): 
    df = pd.read_csv(rf"non_ren_wacc_for_time_-{i}y.csv")
    segment_data = pd.read_csv(rf"segment_numbers/no_ren_segment_numbers_for_time_{-i}y.csv")
    df = df.merge(segment_data, left_on="Firm Name", right_on="Instrument")
    #drop insrtument colum 
    df = df.drop(columns=["Instrument"])
    #remove duplicaets of firm name 
    df = df.drop_duplicates(subset=["Firm Name"])


    coc[i] = df


In [6]:
#get all cocs to a same dataframe 
coc_df = pd.concat(coc.values(), ignore_index=True)
coc_df.to_csv(rf"ren_alternative\no_ren_coc.csv", index=False)

In [ ]:
coc_df

,Firm Name,WACC,COE,COD,year,has ren segment,number of segments
0,000539.SZ,NaN,NaN,0.028785,0,False,1
1,000629.SZ,0.042838,0.056250,0.032144,0,False,5
2,002545.SZ,0.086633,0.126500,0.031242,0,False,5
3,071320.KS,0.017861,0.060703,0.017506,0,True,3
4,2083.SE,0.039836,0.086875,0.024499,0,False,7
...,...,...,...,...,...,...,...
1399,TTPW.NS,0.043682,0.091250,0.078243,-19,False,3
1400,UN01.DE,0.007866,0.157500,0.034612,-19,False,0
1401,UNACEMC1.LM,0.068621,0.121500,0.047682,-19,False,0
1402,VERB.VI,0.029349,0.069336,0.017110,-19,False,5
